## Creating alert packets from HiTS processing

In [26]:
import os
import numpy as np
from io import BytesIO
#%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3
import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.geom
import lsst.alert.packet
afwDisplay.setDefaultBackend('matplotlib')
import matplotlib.ticker as plticker
from astropy.visualization import (ZScaleInterval, SqrtStretch, ImageNormalize)

In [2]:
! ls -lth /project/mrawls/hits2015/rerun/ | head

total 0
drwxrwsr-x 89 52919 lsst_users 4.0K Feb 25 13:08 cw_2020_02
drwxrwsr-x 89 52919 lsst_users 4.0K Jan 30 05:19 cw_2020_01
drwxrwsr-x 89 52919 lsst_users 4.0K Jan  2 07:19 cw_2019_12v2
drwxrwsr-x 89 52919 lsst_users 4.0K Dec 20 09:47 cw_2019_12
drwxrwsr-x 89 52919 lsst_users 4.0K Dec 13 22:41 coaddtest7
drwxrwsr-x 89 52919 lsst_users 4.0K Dec 13 03:59 coaddtest8
drwxrwsr-x 89 52919 lsst_users 4.0K Dec 13 01:03 coaddtest9
drwxrwsr-x 89 52919 lsst_users 4.0K Nov 28 02:31 coaddtest6
drwxrwsr-x 89 52919 lsst_users 4.0K Nov 27 23:46 coaddtest5


In [3]:
# Meredith's standard processing location
REPO = '/project/mrawls/hits2015/rerun/cw_2020_01'

In [4]:
connection = sqlite3.connect(f'{REPO}/association.db')


In [5]:
pd.read_sql_query('select count(diaSourceId) from DiaSource;', connection)


,count(diaSourceId)
0,634915


In [6]:
dia_sources = pd.read_sql_query('select * from DiaSource LIMIT 30;', connection)

In [7]:
dia_sources.head()

,diaSourceId,ccdVisitId,diaObjectId,ssObjectId,parentDiaSourceId,prv_procOrder,ssObjectReassocTime,midPointTai,ra,raErr,...,ixxPSF,iyyPSF,ixyPSF,extendedness,spuriousness,flags,pixelId,filterName,filterId,isDipole
0,176486665823453487,41091504,176486665823453487,0,0,0,None,57070.141866,149.505905,None,...,4.532675,3.881223,0.207817,None,None,25299404,0,g,1,0
1,176486665823453488,41091504,176486665823453488,0,0,0,None,57070.141866,149.506049,None,...,4.532675,3.881223,0.207817,None,None,25299404,0,g,1,0
2,176486665823453489,41091504,176486665823453489,0,0,0,None,57070.141866,149.511464,None,...,4.532675,3.881223,0.207817,None,None,25166152,0,g,1,0
3,176486665823453490,41091504,176486665823453490,0,0,0,None,57070.141866,149.511534,None,...,4.532675,3.881223,0.207817,None,None,25168328,0,g,1,0
4,176486665823453491,41091504,176486665823453491,0,0,0,None,57070.141866,149.512223,None,...,4.532675,3.881223,0.207817,None,None,41943040,0,g,1,1


In [8]:
dia_objects = pd.read_sql_query('select * from diaObject LIMIT 30;', connection)

In [9]:
dia_objects.head()

,diaObjectId,validityStart,validityEnd,ra,raErr,decl,declErr,ra_decl_Cov,radecTai,pmRa,...,yPSFluxPercentile75,yPSFluxPercentile95,yPSFluxMin,yPSFluxMax,yPSFluxStetsonJ,yPSFluxLinearSlope,yPSFluxLinearIntercept,yPSFluxMaxSlope,yPSFluxErrMean,parent
0,176486665823453487,2015-02-17 03:24:17.200177,2015-02-18 03:19:20.963001,149.505905,None,3.015901,None,None,57070.141866,None,...,None,None,None,None,None,None,None,None,None,None
1,176486665823453488,2015-02-17 03:24:17.200177,2015-02-19 03:14:27.983084,149.506049,None,3.018367,None,None,57070.141866,None,...,None,None,None,None,None,None,None,None,None,None
2,176486665823453489,2015-02-17 03:24:17.200177,2015-02-17 05:27:34.070729,149.511464,None,2.886295,None,None,57070.141866,None,...,None,None,None,None,None,None,None,None,None,None
3,176486665823453490,2015-02-17 03:24:17.200177,2015-02-17 05:27:34.070729,149.511534,None,2.883162,None,None,57070.141866,None,...,None,None,None,None,None,None,None,None,None,None
4,176486665823453491,2015-02-17 03:24:17.200177,2015-02-17 05:27:34.070729,149.512223,None,2.932614,None,None,57070.141866,None,...,None,None,None,None,None,None,None,None,None,None


In [10]:
dia_objects.columns

Index(['diaObjectId', 'validityStart', 'validityEnd', 'ra', 'raErr', 'decl',
       'declErr', 'ra_decl_Cov', 'radecTai', 'pmRa',
       ...
       'yPSFluxPercentile75', 'yPSFluxPercentile95', 'yPSFluxMin',
       'yPSFluxMax', 'yPSFluxStetsonJ', 'yPSFluxLinearSlope',
       'yPSFluxLinearIntercept', 'yPSFluxMaxSlope', 'yPSFluxErrMean',
       'parent'],
      dtype='object', length=197)

In [11]:
pd.read_sql_query('select diaSourceId, diaObjectId, count(diaObjectId) from diaSource GROUP BY diaObjectId ORDER BY count(diaObjectId) DESC LIMIT 10;', connection)

,diaSourceId,diaObjectId,count(diaObjectId)
0,176492665892766091,176492665892766091,56
1,176492674482700526,176492674482700526,56
2,176492674482700534,176492674482700534,56
3,176492674482700538,176492674482700538,56
4,176492674482700543,176492674482700543,56
5,176492674482700564,176492674482700564,56
6,176492683072635079,176492683072635079,56
7,176492683072635108,176492683072635108,56
8,176492683072635132,176492683072635132,56
9,176492687367602370,176492687367602370,56


In [12]:
#pd.read_sql_query('select * from diaObject LIMIT 30;', connection)

In [13]:
schema = lsst.alert.packet.Schema.from_file('/project/ebellm/sample-avro-alert/schema/2/1/lsst.alert.avsc')

In [14]:
schema.definition

{'type': 'record',
 'name': 'lsst.alert',
 'fields': [{'doc': 'unique alert identifer',
   'name': 'alertId',
   'type': 'long'},
  {'name': 'diaSource',
   'type': {'type': 'record',
    'name': 'lsst.alert.diaSource',
    'fields': [{'name': 'diaSourceId', 'type': 'long'},
     {'name': 'ccdVisitId', 'type': 'long'},
     {'default': None, 'name': 'diaObjectId', 'type': ['null', 'long']},
     {'default': None, 'name': 'ssObjectId', 'type': ['null', 'long']},
     {'default': None, 'name': 'parentDiaSourceId', 'type': ['null', 'long']},
     {'name': 'midPointTai', 'type': 'double'},
     {'name': 'filterName', 'type': 'string'},
     {'name': 'programId', 'type': 'int'},
     {'name': 'ra', 'type': 'double'},
     {'name': 'decl', 'type': 'double'},
     {'default': None,
      'name': 'ra_decl_Cov',
      'type': ['null',
       {'type': 'record',
        'name': 'lsst.alert.ra_decl_Cov',
        'fields': [{'name': 'raSigma', 'type': 'float'},
         {'name': 'declSigma', 'type'

In [15]:
def get_current_diaObject(diaObjectId, connection):
    df =  pd.read_sql_query(f'select * from diaObject where diaObjectId = {diaObjectId} and validityEnd IS NULL', connection)
    assert(len(df) == 1)
    return df.to_dict('records')[0]

In [16]:
def get_past_diaSources_from_diaObjectId(diaObjectId, connection):
    df =  pd.read_sql_query(f'select * from diaSource where diaObjectId = {diaObjectId}', connection)
    return df.to_dict('records')

In [17]:
#get_past_diaSources_from_diaObjectId(176492665892766091, connection)

In [18]:
ds = dia_sources.iloc[0]

In [21]:
def patchDiaSource(ds):
    #diaSource fixes
    ds['programId'] = 0
    
    required = ['apFlux', 'apFluxErr', 'snr', 'psFlux', 'psFluxErr']
    for require in required:
        if ds[require] is None:
            ds[require] = np.nan

    return ds

def make_alert_json(diaSource, connection):
    alert = {}
    alert['alertId'] = 0
    alert['diaSource'] = patchDiaSource(diaSource.to_dict())
    
    diaObjectId = diaSource['diaObjectId']
    currentDiaSourceId = diaSource['diaSourceId']
    pastDiaSources = get_past_diaSources_from_diaObjectId(diaObjectId, connection)
    alert['prvDiaSources'] = [patchDiaSource(ds) for ds in pastDiaSources if ds['diaSourceId'] != currentDiaSourceId]
    
    alert['prvDiaForcedSources'] = None
    alert['prvDiaNondetectionLimits'] = None
    
    diaObject = get_current_diaObject(diaObjectId, connection)
    alert['diaObject'] = diaObject
    
    alert['ssObject'] = None
    alert['cutoutDifference'] = None
    alert['cutoutTemplate'] = None
    
    return alert

# schema.validate doesn't like the nested schemas but they seem to serialize fine...
#schema.validate(make_alert_json(ds, connection))
schema.deserialize(schema.serialize(make_alert_json(ds, connection)))
#jn = make_alert_json(ds, connection)

{'alertId': 0,
 'diaSource': {'diaSourceId': 176486665823453487,
  'ccdVisitId': 41091504,
  'diaObjectId': 176486665823453487,
  'ssObjectId': 0,
  'parentDiaSourceId': 0,
  'midPointTai': 57070.141865742786,
  'filterName': 'g',
  'programId': 0,
  'ra': 149.5059050242644,
  'decl': 3.015901421494416,
  'ra_decl_Cov': None,
  'x': 165.0045623779297,
  'y': 30.108213424682617,
  'x_y_Cov': None,
  'apFlux': nan,
  'apFluxErr': nan,
  'snr': nan,
  'psFlux': nan,
  'psFluxErr': nan,
  'psRa': None,
  'psDecl': None,
  'ps_Cov': None,
  'psLnL': None,
  'psChi2': None,
  'psNdata': 0,
  'trailFlux': None,
  'trailRa': None,
  'trailDecl': None,
  'trailLength': None,
  'trailAngle': None,
  'trail_Cov': None,
  'trailLnL': None,
  'trailChi2': None,
  'trailNdata': 0,
  'dipMeanFlux': nan,
  'dipFluxDiff': nan,
  'dipRa': None,
  'dipDecl': None,
  'dipLength': nan,
  'dipAngle': nan,
  'dip_Cov': None,
  'dipLnL': None,
  'dipChi2': nan,
  'dipNdata': 0,
  'totFlux': -568.2145385742188

In [23]:
# hacked up from https://github.com/lsst-dm/ap_pipe-notebooks/blob/master/plotLightcurve.py

def get_cutouts(diaSource, repo=REPO, diffimType='deepDiff_differenceExp'):
    centerSource = lsst.geom.SpherePoint(diaSource['ra'], diaSource['decl'], lsst.geom.degrees)
    size = lsst.geom.Extent2I(30, 30)
    
    butler = dafPersist.Butler(repo)
    dataId = diaSource['ccdVisitId']
    visit = int(str(dataId)[0:6])
    ccdnum = int(str(dataId)[6:])
    dataIdDict = {'visit': visit, 'ccdnum': ccdnum}
    diffimFirst = butler.get(diffimType, dataIdDict)
    cutoutDiff = diffimFirst.getCutout(centerSource, size)
    
    return cutoutDiff

cutout= get_cutouts(ds)

In [24]:
help(cutout.writeFits)

Help on method writeFits in module lsst.afw.image.exposure.exposure:

writeFits(...) method of lsst.afw.image.exposure.exposure.ExposureF instance
    writeFits(*args, **kwargs)
    Overloaded function.
    
    1. writeFits(self: lsst.afw.image.exposure.exposure.ExposureF, arg0: str) -> None
    
    2. writeFits(self: lsst.afw.image.exposure.exposure.ExposureF, arg0: lsst.afw.fits.fits.MemFileManager) -> None
    
    3. writeFits(self: lsst.afw.image.exposure.exposure.ExposureF, arg0: lsst.afw.fits.fits.Fits) -> None
    
    4. writeFits(self: lsst.afw.image.exposure.exposure.ExposureF, filename: str, imageOptions: lsst.afw.fits.fits.ImageWriteOptions, maskOptions: lsst.afw.fits.fits.ImageWriteOptions, varianceOptions: lsst.afw.fits.fits.ImageWriteOptions) -> None
    
    5. writeFits(self: lsst.afw.image.exposure.exposure.ExposureF, manager: lsst.afw.fits.fits.MemFileManager, imageOptions: lsst.afw.fits.fits.ImageWriteOptions, maskOptions: lsst.afw.fits.fits.ImageWriteOptions, va

In [48]:
import tempfile
temp = tempfile.NamedTemporaryFile()
#cutout.writeFits('tmp')
cutout.writeFits(temp.name)
with open(temp.name,'rb') as f:
    cutout_bytes = f.read()
os.unlink(temp.name)

In [50]:
#cutout_bytes

In [31]:
from astropy.io import fits

In [51]:
hdulist = fits.open(BytesIO(cutout_bytes))

In [52]:
hdulist

[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x7ff16bd4a6d0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7ff16bd92850>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7ff16bce69d0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x7ff16bcf2190>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7ff16bcf8410>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7ff16bcfd090>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7ff16bd02050>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7ff16bd08110>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7ff16bd08b90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7ff16bd0ec90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7ff16bc24110>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7ff16bc2a610>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7ff16bc34f90>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7ff16bc24fd0>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x7ff16bc24ed0>, <astropy.io.fits.hd